# 受信時間調整用ノートブック

## ライブラリのインポート

In [5]:
import json
import pandas as pd
from tqdm import tqdm

from lib import Util

## 設定ファイルの読み込み

In [6]:
# 設定ファイルの読み込み
with open(f"{Util.get_root_dir()}/../config/config.json", "r") as f:
    config = json.load(f)

## 受信時間調整アルゴリズム

In [ ]:
class TimeAdjuster:
    def __init__(self, df: pd.DataFrame, alpha: float):
        self.df    = df    # データフレーム
        self.alpha = alpha # 補正係数

## 受信時間調整

In [7]:
# 共通ファイルを取得
common_file = Util.get_common_files(path_list=[f"{Util.get_root_dir()}/../data/csv-data/{field_device}/amp/" for field_device in config["FieldDevice"]["Pcap"]])

In [ ]:
# 各ファイルに対して受信時間調整アルゴリズムを適用